In [1]:
import os
import sys 

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:/Users/JSM")\
    .getOrCreate()

In [2]:
#분포 생성
df =spark.range(0, 10) #id란 컬럼명으로 0~9를 입력
df.show()
df.select('id')

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



DataFrame[id: bigint]

In [3]:
from pyspark.sql.functions import rand, randn
colUniform = rand(seed=10).alias("uniform") #rand() 균등분포(랜덤)
colNormal = randn(seed=27).alias("normal") #randn() 정규분포(랜덤) 
df3 = df.select("id", colUniform, colNormal) #select 함수로 컬럼을 포함시킬 수 있다.
df3.show()

+---+-------------------+-------------------+
| id|            uniform|             normal|
+---+-------------------+-------------------+
|  0|0.41371264720975787| 0.5888539012978773|
|  1| 0.7311719281896606| 0.8645537008427937|
|  2| 0.9031701155118229| 1.2524569684217643|
|  3|0.09430205113458567| -2.573636861034734|
|  4|0.38340505276222947| 0.5469737451926588|
|  5| 0.5569246135523511|0.17431283601478723|
|  6| 0.4977441406613893|-0.7040284633147095|
|  7| 0.2076666106201438| 0.4637547571868822|
|  8| 0.9571919406508957|  0.920722532496133|
|  9| 0.7429395461204413|-1.4353459012380192|
+---+-------------------+-------------------+



In [4]:
#기본 통계
df3.describe().show()

+-------+------------------+-------------------+--------------------+
|summary|                id|            uniform|              normal|
+-------+------------------+-------------------+--------------------+
|  count|                10|                 10|                  10|
|   mean|               4.5| 0.5488228646413278|0.009861721586543392|
| stddev|3.0276503540974917| 0.2856822245344392|  1.2126061129356596|
|    min|                 0|0.09430205113458567|  -2.573636861034734|
|    max|                 9| 0.9571919406508957|  1.2524569684217643|
+-------+------------------+-------------------+--------------------+



In [5]:
#freqItems()

#홀수행일 경우 (1, 2, 3) 아닐 경우 (i번쨰 행, 2*i, i%4)
df=spark.createDataFrame([(1, 2, 3) if i%2 == 0 else (i, 2*i, i%4) for i in range(100)], ["a","b","c"])
df.show(10)

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
|  5| 10|  1|
|  1|  2|  3|
|  7| 14|  3|
|  1|  2|  3|
|  9| 18|  1|
+---+---+---+
only showing top 10 rows



In [6]:
freq = df.stat.freqItems(["a", "b", "c"], 0.6) #60% 이상 발생한 숫자(아이템)를 반환합니다
print freq.show()

+-----------+-----------+-----------+
|a_freqItems|b_freqItems|c_freqItems|
+-----------+-----------+-----------+
|        [1]|        [2]|        [3]|
+-----------+-----------+-----------+

None
